In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

plt.rcParams['figure.figsize'] = (4, 4)
plt.rcParams['figure.dpi'] = 150
plt.rcParams['lines.linewidth'] = 3
sns.set()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

### Part 1: Using Regularization and Cross Validation to Avoid Overfitting

Below, we split the NBA data into a training and a test set.

In [ ]:
np.random.seed(5)
df = pd.read_csv("nba.csv").sample(frac = 1)
df["WON"] = df["WL"]
df["WON"] = df["WON"].replace("W", 1)
df["WON"] = df["WON"].replace("L", 0)
df_train, df_test = np.split(df, [2000])

#### P1A: Training an Unregularized Model

Let's use all of the numerical features available to us. These are given in `p1_features` below.

In [ ]:
p1_features = ['FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 
               'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 
               'BLK', 'TOV', 'PF', 'PTS']

Below, train an unregularized LogisticRegression model called `unregularized_model`. Your model should try to fit `df_train["WON"]` from the features in `p1_features`.

Make sure to set `penalty = 'none'`. Also, don't use `PolynomialFeatures`.

In [ ]:
unregularized_model = ...

Below, we compute the accuracy of your classifier on the training and test set using sklearn's built-in `accuracy_score`. You should get around 84% for the training accuracy and around 83% for the test accuracy.

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(df_train["WON"], unregularized_model.predict(df_train[p1_features]))

In [ ]:
accuracy_score(df_test["WON"], unregularized_model.predict(df_test[p1_features]))

#### P1B: Computing the Cross Entropy Loss

Above, we gave the accuracy for your model. Below, compute the average cross-entropy loss on the training and test sets. 

Hint: You'll need to import `log_loss`. 

Hint: `log_loss` needs probabilities, not classes, so make sure to use `predict_proba` instead of `predict`.

You should get around 0.34 for the training set and 0.355 for the test set.

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
training_mean_cross_entropy_loss = ... 

In [ ]:
test_mean_cross_entropy_loss = ...

**Important note:** The `.fit` method in `LogisticRegression` picks parameters that minimize the log loss. It does not directly consider accuracy.

We will see in optional problem 3 that sometimes these two ideas are in conflict.

#### P1C: Training a Degree 3 Unregularized Model

Below, train a model called `unregularized_poly3_model` that is exactly like your model from part 1A, except that it uses `PolynomialFeatures` with degree 3.

In [ ]:
unregularized_poly3_model = ...

Now compute the accuracy of your model on the training and test sets. You should get 100% accuracy training set, and around 73% on your test set.

In [ ]:
accuracy_score(df_train["WON"], unregularized_poly3_model.predict(df_train[p1_features]))

In [ ]:
accuracy_score(df_test["WON"], unregularized_poly3_model.predict(df_test[p1_features]))

We are clearly overfitting here.

#### P1D: Regularized Model with Hard-Coded C

One way to avoid overfitting is to regularize our model. Below, train a model `C1000_poly3_model` that appleis regularization with C1000. Recall, to enable regularization, simply delete `penalty = 'none'`.

In [ ]:
C1000_poly3_model = ...

Give the accuracy on the training and test set below. You should see that the accuracy score is still 100% for the training set, but the test set accuracy is somewhat better at around 75.5%.

In [ ]:
accuracy_score(df_train["WON"], C1000_poly3_model.predict(df_train[p1_features]))

In [ ]:
accuracy_score(df_test["WON"], C1000_poly3_model.predict(df_test[p1_features]))

#### P1E: Using Cross Validation

Above, we selected an arbitrary C. Perhaps some other C would work better. There are two ways to find a better C:

#1. Find the C that works best on a validation set.
#2. Use cross validation.

In this case, we have not kept any data aside as a validation set, so we can't do #1. So let's do #2 instead.

Below train a LogisticRegressionCV model called `cv_poly3_model`, and test C values between 10^-7 and 10^5. For example, you might specify `Cs = 10**np.linspace(-7, 5, 100)`. Set `cv = 3` so that you have 3-fold cross validation.

This might take a while to run.

In [ ]:
cv_poly3_model = ...

We can see what `C` our model picked using cross validation by looking at:

In [ ]:
cv_poly3_model.named_steps["model"].C_

Below, compute the training and test set accuracy for `cv_poly3_model`. On my computer, I got 85% on the training set, and 81% on the test set.

We can see that while our model does not perform as well on the training set, it generalizes much better to our unseen test set than our original model `unregularized_poly3_model`, which only was able to achieve 73% accuracy.

**Bottom line:** `cv_poly3_model` does the best job, and it is able to do this by applying regularization, with hyperparameter C selected through cross validation.

In [ ]:
accuracy_score(df_train["WON"], cv_poly3_model.predict(df_train[p1_features]))

In [ ]:
accuracy_score(df_test["WON"], cv_poly3_model.predict(df_test[p1_features]))

#### Bonus Problem 1F: L1 Regularization (OPTIONAL)

Just like linear regression, we can also use L1 regularization. That is, instead of adding a penalty term that looks like $\sum \theta_i^2$, it looks like $\sum |\theta_i|$.

In HW4, bonus problem 2H, we explored this. Create a model `p1e_model` below that is just like `cv_poly3_model`, except that it uses L1 regularization. The result is that most parameters will be zero.

To use L1 regularization, set `penalty = l1` and set `solver = 'liblinear'`. 

This might several minutes to finish running, depending on your computer. Consider reducing the number of Cs if you don't want to wait that long.

In [ ]:
p1f_model = ...

In [ ]:
p1f_model.named_steps["model"].C_

In [ ]:
p1f_model.named_steps["model"].intercept_

In [ ]:
p1f_model.named_steps["model"].coef_

Below, similar to problem 2H from exercise 4, create a table of all non-zero coefficients along with their names. You should have around 100 non-zero features, out of a total of 1,330.

### Part 2: Logistic Regression from Scratch (Optional)

The code below fits an sklearn `LogisticRegression` model with `C = 1e-2` to the NBA dataset.

In [ ]:
p2_features = ['FGM', 'FGA', 'STL']
p2_sklearn_model = LogisticRegression(C = 1e-3)
p2_sklearn_model.fit(df_train[p2_features], df_train["WON"])

In [ ]:
p2_sklearn_model.intercept_

In [ ]:
p2_sklearn_model.coef_

Using this model, we can predict the chance that a team will win if it attempts 50 field goals, succeeds at 30 of them, and also performs 20 steals. In this case, the model predicts this team will win with approximately 84.8% probability.

In [ ]:
p2_sklearn_model.predict_proba([[30, 50, 20]])

We can also make multiple predictions at once, i.e. below we see that a team that makes 0 out of 0 field goal attempts and also makes 0 steals is predicted to win with an approximately 9.9% chance by our model.

In [ ]:
p2_sklearn_model.predict_proba([[30, 50, 20], [0, 0, 0]])

In problem 1, we saw how to fit such models. In this problem, we'll explore how the `LogisticRegression` class really works.

#### P1A: Understanding `predict_proba`

Below, implement the `my_predict_proba` function.

This function should return the chance that each of the given teams wins the game as a function of intercept and coefficient.

In [ ]:
def my_predict_proba(intercept, coef, X):
    ...

For example, when running the code below, you should get back `[0.848, 0.0933]`.

In [ ]:
p1a_intercept = -2.20493073
p1a_coef = np.array([0.19994998, -0.07420132,  0.08193321])
p1a_X = np.array([[30, 50, 20],
                  [0, 0, 0]])
my_predict_proba(p1a_intercept, p1a_coef, p1a_X)

#### P2B: Cross Entropy Loss Review

Suppose that we have two basketball games:
1. Team A makes 30 out of 50 field goals, and steals the ball 20 times. Team A wins.
2. Team A makes 0 out of 0 field goals, and steals the ball 0 times. Team A loses.

Write code below that computes the **total** cross-entropy loss of our model above on the given data. You can either do this by implementing the cross-entropy loss function yourself, or by using the sklearn `log_loss` function.

**Hint:** If you set the parameter `normalize = False` in the sklearn `log_loss` function, you will get the total cross-entropy loss. 

You should get a value around 0.269.

In [ ]:
#your code here

#### P2C: Implementing `p2c_total_cross_entropy_loss`

Below, define a function `p2c_total_cross_entropy_loss` that gives the cross entropy loss of the model with given `coef` and `intercept` applied to the given `X` and `y` data.

**Hint:** You'll need to use the `my_predict_proba` function.

In [ ]:
def p2c_total_cross_entropy_loss(y, intercept, coef, X):
    ...

The code below should give the same answer as you computed in `P2B`. In other words, it should be approximatley 0.269.

In [ ]:
p2c_total_cross_entropy_loss([1, 0], p1a_intercept, p1a_coef, p1a_X)

Below, we can compute the total cross entropy loss of the same model on the entire NBA dataset. You should get around 1,158.

In [ ]:
p2c_total_cross_entropy_loss(df_train["WON"], p1a_intercept, p1a_coef, df_train[p2_features])

#### P2D: Getting a Better Fit Manually

Try changing the intercept and coefficients provided as parameters to the function below. Observe what happens to the total cross entropy loss.

Most likely, if you change any values, you'll get slightly worse results. This is because the intercept and coefficients that we are using have been generated by sklearn to be optimal for this dataset.

In [ ]:
p2c_total_cross_entropy_loss(df_train["WON"], -2.20493073, np.array([[0.19994998, -0.07420132,  0.08193321]]),
                             df_train[p2_features])

#### P2E: Understanding `minimize`

Consider the function `x^4 - 4*x^3 + 2*x^2 + x + 1`. The minimum of this function can be seen visually:

In [ ]:
x = np.linspace(-2, 4, 100)
def f_example_1D(x):
    return x**4 - 4*x**3 + 2*x**2 + x + 1

plt.plot(x, f_example_1D(x));

The `minimize` function of sklearn is able to find the minima of arbitrary functions. For example:

In [ ]:
from scipy.optimize import minimize
minimize(f_example_1D, x0 = [1.3])

The code above tells us that the minimizing x is 2.57. We can extract this optimal value by asking for the 'x' from the above dictionary.

In [ ]:
best_x = minimize(f_example_1D, x0 = [1.3])['x']
print(best_x)

The `x0` we provide to minimize tells the scipy optimizer where to start looking for an optimum. That is, we can think of our algorithm as starting at the red dot and eventually ending up at the minimum at the blue dot.

In [ ]:
x = np.linspace(-2, 4, 100)
def f_example_1D(x):
    return x**4 - 4*x**3 + 2*x**2 + x + 1

plt.plot(x, f_example_1D(x))
plt.plot(1.3, f_example_1D(1.3), 'ro', markersize = 20)
plt.plot(2.57, f_example_1D(2.57), 'bo', markersize = 20);

**Technical Note:** For general functions, the starting point we select can affect the final outcome. However, the cross entropy loss we'll be using later in this problem has a special property called "convexity" where the starting point we select does not matter. This is beyond the scope of our class.

We can also use `optimize` to minimize multi-dimensional functions. For example, consider the function (1 - x)^2 + 100(y - x^2 + 2)^2, which you can see visualized below or at [this link](https://en.wikipedia.org/wiki/Rosenbrock_function).

In [ ]:
from mpl_toolkits import mplot3d

def f(p):
    x = p[0]
    y = p[1]
    return (1 - x)**2 + 100*(y - x**2 + 2)**2

X = np.linspace(-2, 2, 50)
Y = np.linspace(-2, 2, 50)

p = np.meshgrid(X, Y)
Z = f(p)

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.contour3D(X, Y, Z, 50, cmap='binary')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z');

The function is smallest in the large flat light colored region. Let's see what happens when we use scipy optimize on this function. We'll start at x = 0, y = 2, which is at the top of the black hill.

In [ ]:
minimize(f, x0 = [0, 2])

This time, we see the optimal input is approximately x = 1, y = -1. We can again plot this result as a starting red dot and a final blue dot.

In [ ]:
from mpl_toolkits import mplot3d

def f(p):
    x = p[0]
    y = p[1]
    return (1 - x)**2 + 100*(y - x**2 + 2)**2

X = np.linspace(-2, 2, 50)
Y = np.linspace(-2, 2, 50)

p = np.meshgrid(X, Y)
Z = f(p)

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.contour3D(X, Y, Z, 50, cmap='binary')
ax.scatter(0, 2, 1600, c='r', s = 500)
ax.scatter(1, -1, 0, c='b', s = 500)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z');


Now it's your turn. As an exercise, use scipy's `minimize` to find the minimum of $$x^2 + \frac{y^2}{3} + z^2 + 2x + 5y + 1$$

You should get x = -1, y = -1.5, z = 0.

In [ ]:
#your code here

#### P2F: Fitting the model (Hard Problem!)

Use `minimize` to find `intercept` and `coef` that minimize training error.

**Hint:** Define a function `total_nba_cross_entropy_training_loss(thetas)` that gives the total cross entropy loss on the training set as a function of the intercept and coefficients. 

**Hint:** Your `p2c_total_cross_entropy_loss` problem will be useful here.

Your code shield give approximately intercept = `-2.44` and coefficients = `0.24, -0.09,  0.11`.

In [ ]:
#your code here

The intercept and theta we got above are not the same as we used earlier in the problem. For example, your code from P2E found that the coefficient for `FGA` is 0.24, but the example I gave at the beginning of this problem was 0.2.

The reason is that the coefficient we computed at the beginning of this problem was from a regularized model where `C=1e-3`.

Below, we create a `LogisticRegression` object with `penalty = 'none'` and fit it to the training set.

In [ ]:
p2f_model = LogisticRegression(penalty = 'none')
p2f_model.fit(df_train[p2_features], df_train["WON"])

Observe below that the `intercept_` and `coef_` and they should be very close to the intercept and coefficients that you computed earlier in `P2E`.

This is great! It means that what you just did in `P2E` is implement your very own logistic regression model. Good job!

In [ ]:
p2f_model.intercept_

In [ ]:
p2f_model.coef_

#### P2G: Fitting a Regularized Model

In P2E, you found the $\theta$ values that minimized the total cross entropy loss (TCEL). Or in otherwords, you minimized:

$$\sum_{i = 1}^n -\ln(1-\Delta y_i)$$

The function that sklearn minimizes when regularizing is:

$$\frac{1}{2C} \sum_{i = 2}^d \theta_i^2 + \sum_{i = 1}^n -\ln(1-\Delta y_i)$$

Here we assume the intercept is $\theta_1$, so it is not included in the left sum.

Below, use `minimize` to compute the $\theta$ values that minimize the sum above.

You should get the same $\theta$ we used at the beginning of Problem 2, i.e. intercept = -2.2, and coefficients = 0.2, -0.07, and 0.08.

Once you've done this, you've shown that you've essentially implemented your own `fit` function. Good job! 

In [ ]:
#your code here

#### P2H: Bonus Exercise: Building Your Own LogisticRegression Class (hard)

Given what you learned in the previous parts of this problem, implement the `MyLogisticRegression` class below.

In [ ]:
class MyLogisticRegression(object):
    def __init__(self, C = 1.0):
        self.C = C
        
    def fit(self, X, y):        
        def log_loss_function_plus_penalty(thetas):
            model = MyLogisticRegression()
            model.intercept_ = thetas[0]
            model.coef_ = thetas[1:]            
            return log_loss(y, model.predict_proba(X), normalize = False) + 1.0 / (2 * self.C) * np.sum(thetas[1:]**2)

        
        
        params = minimize(log_loss_function_plus_penalty, x0 = np.zeros(X.shape[1] + 1))['x']
        self.intercept_ = params[0]
        self.coef_ = params[1:]
        
    def predict_proba(self, X):
        """return 1D array equal to probability that X belongs to class 1"""
        def sigma(x):
            return 1 / (1 + np.exp(-x))
            
        return sigma(np.sum(self.coef_ * X, 1) + self.intercept_)
    
    def predict(self, X):
        """returns 1D array of predictions for X"""
        return (self.predict_proba(X) > 0.5).astype(int)

In [ ]:
p2f_model = MyLogisticRegression(C = 1e-3)
p2f_model.fit(df_train[p2_features], df_train["WON"])

The intercept should be -2.2.

In [ ]:
p2f_model.intercept_

Coefficients should be 0.2, -0.07, 0.08/.

In [ ]:
p2f_model.coef_

Predictions should be 0.85, 0.1.

In [ ]:
p2f_model.predict_proba([[30, 50, 20],
                         [0, 0, 0]])

#### Optional Part 3: Intercept Regularization

This part of the homework is less important, but some of you might find it interesitng.

In [ ]:
titanic_df = sns.load_dataset("titanic")
titanic_df = titanic_df[["age", "fare", "pclass", "sex", "survived"]].copy()
titanic_df = titanic_df.dropna()
titanic_df['sex'] = titanic_df['sex'].replace("male", 0)
titanic_df['sex'] = titanic_df['sex'].replace("female", 1)
np.random.seed(23)
titanic_train, titanic_test = np.split(titanic_df, [600])

Below, we have trained two models on the titanic dataset.

As described in lecture, `tiny_c_model_fit_intercept_true` has two intercepts, one of which is regularized, and one of which is unregularized.

In [ ]:
tiny_c_model_fit_intercept_true = Pipeline([        
    ('scale', StandardScaler()),        
    ('poly', PolynomialFeatures(degree=2)),
    ('model', LogisticRegression(fit_intercept = True, C = 1e-8))
])   
tiny_c_model_fit_intercept_true.fit(titanic_train[["fare", "age", "pclass",  "sex"]], 
                                titanic_train["survived"]);

We see that one of the intercepts is around 10^-12. This is 0th entry of the `coef_`.


In [ ]:
tiny_c_model_fit_intercept_true.named_steps["model"].coef_

And the other is around -0.377. This is the unregularized intercept.

In [ ]:
tiny_c_model_fit_intercept_true.named_steps["model"].intercept_

By contrast, if we set `fit_intercept = False`, the resulting `intercept_` is zero, and we're left with only the regularized intercept given by the 0th entry of `coef_`.

In [ ]:
tiny_c_model_fit_intercept_false = Pipeline([        
    ('scale', StandardScaler()),        
    ('poly', PolynomialFeatures(degree=2)),
    ('model', LogisticRegression(fit_intercept = False, C = 1e-8))
])   
tiny_c_model_fit_intercept_false.fit(titanic_train[["fare", "age", "pclass",  "sex"]], 
                                titanic_train["survived"]);

In [ ]:
tiny_c_model_fit_intercept_false.named_steps["model"].coef_

In [ ]:
tiny_c_model_fit_intercept_false.named_steps["model"].intercept_

#### P3A: Accuracy

Since the `tiny_c_model_fit_intercept_true` has more freedom to pick a better intercept, you might expect that it should have higher accuracy. Let's test this hypothesis.

Compute the accuracy of our two models using sklearn's `accuracy_score` function.

In [ ]:
#your code here

In [ ]:
#your code here

You should see that surprisingly, the model with the `fit_intercept = False` actually has higher accuracy! Over the next few problems, we'll see why.

#### P3B: Cross Entropy 

This time, compute the **total** cross entropy loss for the two models. You should see that `tiny_c_model_fit_intercept_true` is better than `tiny_c_model_fit_intercept_false`, i.e. the log_loss is lower for the model that pick its intercept without worrying about regularization. 

You should get approximately 405 for `tiny_c_model_fit_intercept_true` and 416 for `tiny_c_model_fit_intercept_false`.

Reminder; Make sure to set `normalize = False` when calling `log_loss`. This gives you back the total cross entropy loss instead of the average.

In [ ]:
#your code here

In [ ]:
#your code here

Our observations are summarized in the table below.

|                       | accuracy | cross entropy loss |
|-----------------------|----------|--------------------|
| `fit_intercept = True`  | 59%      | 405                |
| `fit_intercept = False` | 77%      | 416                |

Which is better? It depends on how you define better.

The LogisticRegression model defines "better" as lower cross-entropy loss. From that perspective, `fit_intercept = True` is bettre, since cross entropy loss is only 405.

However, as a human using the model, you might care more about accuracy. Or perhaps you care about another measure like "true positive rate". We will discuss other such measurements in a future lecture.

**Note: The REAL problem here is that we have picked a C which is much too small (C = 1e-8). This results in a model that is vastly underfit and cannot perform well on the provided data.** In real world scenarios with reasonable hyperparameter choices, it is uncommon for accuracy and cross entropy loss to be so out of sync.

#### P3C: A Closer Look

You might find it interesting to see the output of our two models:

In [ ]:
tiny_c_model_fit_intercept_true.predict_proba(titanic_train[["fare", "age", "pclass",  "sex"]])

In [ ]:
tiny_c_model_fit_intercept_false.predict_proba(titanic_train[["fare", "age", "pclass",  "sex"]])

What we see is that the `fit_intercept_true` model basically just gives every passenger a 40.6% chance of surviving. This is also the fraction of passengers who survived. That means that the class it predicts for every passenger is 0, i.e. it thinks everyone died. This is why the accuracy is relatively poor, since it gets every single surviving passenger wrong.

By contrast, `fit_intercept_false` gives every passenger an approximately 50% chance of surviving. In this case, the classes for passengers are 0 or 1, resulting in an overall 77% accuracy.